In [ ]:
%pip install transformers torch datasets evaluate accelerate timm kagglehub pandas seaborn

# Génération de blagues en français
Ce notebook montre plusieurs approches pour générer des blagues en français en partant de modèles que vous avez déjà entraînés pour détecter l'humour.

Approches recommandées :
- Prompt engineering : utiliser un modèle causal pré-entraîné (idéalement en français ou multilingue) et concevoir des prompts qui guident la génération.
- Filtrage / reranking : générer plusieurs candidats puis utiliser vos modèles de détection (`humor_detection_model01`, `humor_model_multilingual`) pour scorer et garder les meilleures blagues.
- Fine-tuning : si vous avez un dataset de blagues (`datasets/shortjokes.csv`), fine-tuner un modèle causal sur ce corpus donne généralement de meilleurs résultats stylistiques et linguistiques.

Dans les cellules suivantes :
1) Exemple de génération avec `transformers` (pipeline text-generation)
2) Exemple de filtrage / reranking en chargeant vos classifieurs locaux
3) Squelette pour fine-tuning si vous souhaitez entraîner un modèle génératif sur vos blagues.

In [ ]:
# Notebook: génération de blagues en français
# Pré-requis: transformers, torch, datasets installés (voir la cellule d'en haut)
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

# Choix du modèle: ici on utilise un modèle causal multilingue léger si disponible.
# Remplacez 'gpt2' par le chemin vers un modèle local (ex: './humor_model_multilingual') ou HF id.
model_name_or_path = 'gpt2'  # remplacer par un modèle FR ou local si vous en avez
device = 0 if torch.cuda.is_available() else -1

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
# GPT-2 n'a pas de token pad par défaut, définir pad_token si absent
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
if device == 0:
    model = model.to('cuda')

gen = pipeline('text-generation', model=model, tokenizer=tokenizer, device=device)

# Exemple de fonction de génération simple en français
def generate_jokes(prompt, max_length=60, temperature=1.0, top_p=0.9, num_return_sequences=5):
    full_prompt = prompt if prompt.endswith('\n') else prompt + '\n'
    outputs = gen(full_prompt, max_length=max_length, do_sample=True, temperature=temperature, top_p=top_p, num_return_sequences=num_return_sequences)
    return [o['generated_text'][len(full_prompt):].strip() for o in outputs]

# Exemples d'appels
prompt = 'Génère une blague courte en français :'
jokes = generate_jokes(prompt, max_length=80, temperature=0.9, top_p=0.95, num_return_sequences=8)
for i,j in enumerate(jokes,1):
    print(f'--- Blague {i} ---')
    print(j)
    print()

In [ ]:
# Filtrage / reranking avec vos modèles de détection d'humour locaux (implémentation robuste)
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import numpy as np
import os
import json

# Chemins vers vos modèles locaux (existant dans le repo)
local_model_1 = './humor_detection_model01'
local_model_2 = './humor_model_multilingual'

def load_model_and_tokenizer(path):
    try:
        abs_path = os.path.abspath(path) if path is not None else None
        if abs_path is None or not os.path.exists(abs_path):
            print(f"Le chemin {path} n'existe pas (résolu en {abs_path}).")
            return None, None
        try:
            print(f'Chargement du modèle depuis: {abs_path} -> contenu:', os.listdir(abs_path))
        except Exception:
            pass

        # Charger le modèle (doit fonctionner si config.json + poids présents)
        model = AutoModelForSequenceClassification.from_pretrained(abs_path)

        # Déterminer un tokenizer adapté : essayer d'abord le tokenizer local, sinon choisir selon model_type dans config
        tok = None
        try:
            tok = AutoTokenizer.from_pretrained(abs_path)
        except Exception as e_tok:
            print(f"Pas de tokenizer local trouvé dans {abs_path} (erreur: {e_tok}). Recherche d'un tokenizer adapté selon config...")
            # lire config pour détecter model_type
            cfg_path = os.path.join(abs_path, 'config.json')
            try:
                with open(cfg_path, 'r', encoding='utf-8') as f:
                    cfg = json.load(f)
                    model_type = cfg.get('model_type', '')
            except Exception:
                model_type = ''
            # heuristiques de fallback
            tried = []
            if 'distilbert' in model_type.lower() or any('Distil' in a for a in cfg.get('architectures', [])):
                tried.append('distilbert-base-multilingual-cased')
            tried.extend(['bert-base-multilingual-cased', 'distilbert-base-uncased', 'bert-base-uncased'])
            for candidate in tried:
                try:
                    tok = AutoTokenizer.from_pretrained(candidate)
                    print(f'Utilisation du tokenizer de fallback: {candidate}')
                    break
                except Exception:
                    tok = None
            if tok is None:
                # dernier recours
                tok = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
                print('Utilisation du tokenizer fallback final: bert-base-multilingual-cased')

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model.to(device)
        model.eval()
        return model, tok

    except Exception as e:
        print(f"Erreur chargement {path}: {e}")
        return None, None

model1, tok1 = load_model_and_tokenizer(local_model_1)
model2, tok2 = load_model_and_tokenizer(local_model_2)

def score_jokes_with_model(jokes, model, tokenizer, device=None, max_length=128):
    if model is None or tokenizer is None:
        return [0.0] * len(jokes)
    device = device or (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))
    scores = []
    # id2label si présent
    id2label = getattr(model.config, 'id2label', None)
    for j in jokes:
        try:
            inputs = tokenizer(j, truncation=True, padding=True, max_length=max_length, return_tensors='pt', return_token_type_ids=False)
            inputs = {k: v.to(device) for k, v in inputs.items()}
            with torch.no_grad():
                outputs = model(**inputs)
                logits = outputs.logits
                probs = torch.softmax(logits, dim=-1).cpu().numpy()[0]
            # construire mapping label->prob
            if id2label is not None:
                labels = [id2label[i] for i in range(len(probs))]
            else:
                labels = [f'LABEL_{i}' for i in range(len(probs))]
            probs_dict = {labels[i]: float(probs[i]) for i in range(len(probs))}
            # heuristique pour probabilité d'humour
            h = None
            for key in probs_dict:
                if 'humor' in key.lower() or 'humour' in key.lower():
                    h = probs_dict[key]
                    break
            if h is None:
                h = probs_dict.get('LABEL_1', max(probs_dict.values()))
        except Exception as e:
            print('Erreur scoring:', e)
            h = 0.0
        scores.append(float(h))
    return scores

# Exécuter le scoring et reranking si la variable `jokes` existe
try:
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    scores1 = score_jokes_with_model(jokes, model1, tok1, device)
    scores2 = score_jokes_with_model(jokes, model2, tok2, device)
    combined = [(s1 + s2) / 2.0 for s1, s2 in zip(scores1, scores2)]

    order = np.argsort(combined)[::-1]
    print('Reranking des blagues par score moyen de humour :')
    for idx in order:
        print('--- Blague (score=', round(combined[idx], 3), ') ---')
        print(jokes[idx])
        print()
except NameError:
    print("La variable 'jokes' n'existe pas : exécutez d'abord la cellule de génération pour créer des candidats de blagues.")


## Fine-tuning minimal (squelette)
Si vous souhaitez fine-tuner un modèle causal sur `datasets/shortjokes.csv` :
1. Charger le CSV et construire un dataset texte (une blague par exemple).
2. Tokenizer et préparer les exemples pour le modèle causal (concatenate prompt+target si besoin).
3. Utiliser `transformers.Trainer` ou `accelerate` pour l'entraînement.

Exemple minimal :
```python
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
import numpy as np

ds = load_dataset('csv', data_files='datasets/shortjokes.csv')['train']
# Supposons que la colonne s'appelle 'text'
ds = ds.map(lambda x: {'text': x['text']})
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def tokenize_fn(ex):
    return tokenizer(ex['text'], truncation=True, max_length=128)

tok_ds = ds.map(tokenize_fn, batched=True, remove_columns=ds.column_names)
tok_ds.set_format(type='torch')

training_args = TrainingArguments(output_dir='./fine_tuned_jokes',
                                  per_device_train_batch_size=8,
                                  num_train_epochs=3,
                                  save_steps=500)
model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
trainer = Trainer(model=model, args=training_args, train_dataset=tok_ds)
trainer.train()
```

## Try it (exécution)
- Exécutez les cellules dans l'ordre: installation -> génération -> filtrage.
- Si vous avez des modèles locaux pour la génération, modifiez `model_name_or_path` pour pointer vers le répertoire du modèle.
- Vérifiez que `datasets/shortjokes.csv` contient une colonne `text` ou adaptez le nom de colonne dans le squelette.

---
Si vous voulez, je peux: fournir un script standalone, préparer un fine-tuning complet avec `accelerate`, ou générer un petit jeu d'évaluation pour mesurer la qualité des blagues filtrées.